In [238]:
!pip install transformers pytelegrambotapi

In [239]:
from transformers import pipeline
import telebot
import re
import time
from threading import Thread

API_TOKEN = '7343295231:AAEYfq3zH-HJ-x8WGHLaxQymmqMO2viaZOI'

In [240]:
intents = [
    "iniciar_pedido",
    "iniciar_conversa",
    "pedir_sorvete",
    "pedir_cardapio",
    "info_contato",
    "horario",
    "outro",
    "cancelar_pedido",
    "finalizar_pedido"
]

In [241]:
item_prices = {
    "Sorvete de Açaí": 5.00,
    "Sorvete de CheeseCake": 5.00,
    "Sorvete de Chocolate": 5.00,
    "Sorvete de Flocos": 5.00,
    "Sorvete de Kinder Ovo": 5.00,
    "Sorvete de Morango": 5.00,
    "Sorvete de Napolitano": 5.00,
    "Sorvete de Ninho Trufado": 5.00,
    "Sorvete de Pistache": 5.00,
    "Sorvete de Tapioca": 5.00,
    "Sorvete de Torta de Limão": 5.00,
    "Sorvete de Uva": 5.00,
    "Milkshake de Café": {
        "300ml": 10.00,
        "500ml": 15.00,
        "700ml": 20.00
    },
    "Milkshake de Chocolate": {
        "300ml": 10.00,
        "500ml": 15.00,
        "700ml": 20.00
    },
    "Milkshake de Frutas Vermelhas": {
        "300ml": 12.00,
        "500ml": 18.00,
        "700ml": 24.00
    },
    "Milkshake de Morango": {
        "300ml": 10.00,
        "500ml": 15.00,
        "700ml": 20.00
    },
    "Milkshake de Ovomaltine": {
        "300ml": 12.00,
        "500ml": 18.00,
        "700ml": 24.00
    },
    "Milkshake de Torta de Limão": {
        "300ml": 12.00,
        "500ml": 18.00,
        "700ml": 24.00
    },
    "Picolé de Banana": 3.00,
    "Picolé de Blue Ice": 3.00,
    "Picolé de Chiclete": 3.00,
    "Picolé de Leite Condensado": 3.00,
    "Picolé de Milho Verde": 3.00,
    "Picolé de Morango": 3.00,
    "Picolé de Uva": 3.00
}

In [242]:
def get_item_price(item, size=None):
    if isinstance(item_prices[item], dict):
        return item_prices[item].get(size, "Tamanho não disponível")
    else:
        return item_prices[item]

In [243]:
def summarize_order(order_details):
    total_amount = 0
    order_summary = "*Resumo do Pedido:*\n\n"

    for item, (quantity, size) in order_details.items():
        item_price = get_item_price(item, size)
        total_amount += item_price * quantity
        size_text = f" ({size})" if size else ""
        order_summary += f"{item}{size_text} (x{quantity}): R${item_price * quantity:.2f}\n"

    order_summary += f"\n*Total:* R${total_amount:.2f}"
    return order_summary

In [244]:
sorveteria_latitude = -23.22907
sorveteria_longitude = -45.88675

In [245]:
def handle_payment_choice(message):
    user_id = message.chat.id
    payment_method = message.text.lower()

    if payment_method in ['pix', 'cartão de débito', 'cartão de crédito', 'dinheiro']:
        bot.reply_to(message, "Ok, favor realizar o pagamento no balcão ao ir buscar seu pedido.")
        bot.send_location(user_id, sorveteria_latitude, sorveteria_longitude)
        start_order_preparation(user_id)
    else:
        bot.reply_to(message, "Por favor, escolha uma opção válida de pagamento.")

In [246]:
def send_sorveteria_location(user_id):
    bot.send_message(user_id, "Seu pedido está pronto para ser retirado. Aqui está a localização da sorveteria:")
    bot.send_location(user_id, sorveteria_latitude, sorveteria_longitude)

In [247]:
def start_order_preparation(user_id):
    preparation_time_minutes = 2

    bot.send_message(user_id, f"Tempo restante para o pedido ficar pronto: {preparation_time_minutes:02d}:00")

    for remaining_minutes in range(preparation_time_minutes, 0, -1):
        time.sleep(60)
        bot.send_message(user_id, f"Tempo restante para o pedido ficar pronto: {remaining_minutes - 1:02d}:00")

    bot.send_message(user_id, "Seu pedido está pronto! :)")

In [248]:
bot = telebot.TeleBot(API_TOKEN)

In [249]:
classifier = pipeline('zero-shot-classification', model='facebook/bart-large-mnli')

In [250]:
orders = {}
pending_sizes = {}

In [251]:
def extract_items_and_quantities(text):
    pattern = r"(\d+)?\s*(picolé|sorvete|milkshake) de ([a-z\s]+)(?:\s+e\s+|\s*,\s*|$)"
    matches = re.findall(pattern, text, re.IGNORECASE)
    items = {}
    for match in matches:
        quantity = int(match[0]) if match[0] else 1
        item_type = match[1].capitalize()
        item_flavor = match[2].strip().lower()
        item_flavor = " ".join(word.capitalize() for word in item_flavor.split())
        item = f"{item_type} de {item_flavor}"
        if item in items:
            items[item] += quantity
        else:
            items[item] = quantity
    return items

In [252]:
@bot.message_handler(func=lambda message: True)
def handle_message(message):
    global orders
    user_id = message.chat.id
    text = message.text.lower()

    if user_id in pending_sizes:
        item = pending_sizes.pop(user_id)
        if text in item_prices[item]:
            if user_id not in orders:
                orders[user_id] = {}
            if item in orders[user_id]:
                orders[user_id][item] = (orders[user_id][item][0] + 1, text)
            else:
                orders[user_id][item] = (1, text)
            bot.reply_to(message, f"Adicionei {item} ({text}) ao seu pedido. Você gostaria de mais alguma coisa? Se sim, apenas me avise. Se não, digite 'finalizar'. Se deseja cancelar o pedido, digite 'cancelar'.")
        else:
            bot.reply_to(message, "Tamanho não disponível. Por favor, escolha entre 300ml, 500ml ou 700ml.")
        return

    # Identificação de intenções usando palavras-chave
    if "cardápio" in text or "menu" in text:
        intent = "pedir_cardapio"
    elif any(keyword in text for keyword in ["quero", "vou querer", "gostaria de"]):
        intent = "pedir_sorvete"
    elif "finalizar" in text:
        intent = "finalizar_pedido"
    elif "cancelar" in text:
        intent = "cancelar_pedido"
    else:
        result = classifier(text, intents)
        intent = result['labels'][0]

    if intent == "iniciar_conversa":
        bot.reply_to(message, "Olá! Bem-vindo à Sorveteria. Como posso ajudá-lo hoje?")
    elif intent == "iniciar_pedido":
        bot.reply_to(message, "Iniciando um novo pedido. Por favor, escolha os itens que deseja adicionar ao seu carrinho.")
        if user_id in orders:
            orders[user_id] = {}
    elif intent == "pedir_cardapio":
        cardapio = "\n".join([f"• {item} - R${preco:.2f}" if not isinstance(preco, dict) else f"• {item} - (consulte os tamanhos)" for item, preco in item_prices.items()])
        bot.reply_to(message, f"Os sabores disponíveis são:\n{cardapio}")
    elif intent == "pedir_sorvete":
        items = extract_items_and_quantities(text)
        if not items:
            bot.reply_to(message, "Desculpe, não entendi qual item você gostaria de pedir.")
        else:
            for item, quantity in items.items():
                if item in item_prices:
                    if isinstance(item_prices[item], dict):
                        pending_sizes[user_id] = item
                        bot.reply_to(message, "Qual tamanho você gostaria? (300ml, 500ml, 700ml):")
                    else:
                        if user_id not in orders:
                            orders[user_id] = {}
                        if item in orders[user_id]:
                            orders[user_id][item] = (orders[user_id][item][0] + quantity, None)
                        else:
                            orders[user_id][item] = (quantity, None)
                        bot.reply_to(message, f"Adicionei {quantity} {item}(s) ao seu pedido. Você gostaria de mais alguma coisa? Se sim, apenas me avise. Se não, digite 'finalizar'. Se deseja cancelar o pedido, digite 'cancelar'.")
                else:
                    bot.reply_to(message, f"Desculpe, não temos {item}.")
    elif intent == "outro":
        bot.reply_to(message, "Desculpe, não entendi sua solicitação.")
    elif intent == "cancelar_pedido":
        if user_id in orders:
            orders.pop(user_id)
            bot.reply_to(message, "Seu pedido foi cancelado.")
        else:
            bot.reply_to(message, "Você não tem nenhum pedido em andamento.")
    elif intent == "finalizar_pedido":
        if user_id in orders and orders[user_id]:
            order_summary = summarize_order(orders[user_id])
            bot.reply_to(message, order_summary)
            bot.reply_to(message, "Qual o método de pagamento desejado?\n• Pix\n• Cartão de débito\n• Cartão de crédito\n• Dinheiro")
            bot.register_next_step_handler(message, handle_payment_choice)
            orders.pop(user_id)
        else:
            bot.reply_to(message, "Você não tem nenhum pedido em andamento para finalizar.")
    elif text in item_prices:
        if isinstance(item_prices[text], dict):
            pending_sizes[user_id] = text
            bot.reply_to(message, "Qual tamanho você gostaria? (300ml, 500ml, 700ml):")
        else:
            if user_id not in orders:
                orders[user_id] = {}
            if text in orders[user_id]:
                orders[user_id][text] = (orders[user_id][text][0] + 1, None)
            else:
                orders[user_id][text] = (1, None)
            bot.reply_to(message, f"Adicionei {text} ao seu pedido. Você gostaria de mais alguma coisa? Se sim, apenas me avise. Se não, digite 'finalizar'. Se deseja cancelar o pedido, digite 'cancelar'.")
    else:
        bot.reply_to(message, "Desculpe, não entendi sua solicitação.")


In [ ]:
bot.polling()